# Begin

Here are all the prerequisite files that need to be imported

In [ ]:
import Email_Sender
import Platform_API
import Platform_Alert
import SQL_Caller
import Platform_Objects
from datetime import datetime

Import main so this page is correct, do not import main or use main.xx if code is in main:

In [ ]:
import main

To connect to the database (of course, locally setup it first), run the command in main:

In [ ]:
host = "localhost"
user = "pi"
password = "2333"
database = "prototype"
global mydb, cursor
mydb, cursor = SQL_Caller.establish(user, password, host, database)

# DISASTER

1.Search Disaster by id, display detailed data of the event.
Function input: id
Return: detailed data in any form (Prefer list)

In [ ]:
test_disaster = Platform_Objects.Disaster(d_id='01aeec6d-4577-437c-a02c-649330d88bf3')
test_disaster.load_from_database(mydb, cursor)

* After this, all the necessary data are in the test_disaster that can be accessed with things like "test_disaster.area" and such, more detailed attributes are in the "Objects" file.

2.Search by State, display list of most recent 50 disasters' alert of that state
Function input: state
Return: list of most recent 50 disasters' alert of that state

In [ ]:
result = SQL_Caller.disaster_top_50("CA")

3.Search by State and County, display list of most recent 50 disasters' alert of that county
Function input: state
Return: list of most recent 50 disasters' alert of that county in that state

* Did not code due to the data indicates no necessity to use more detail.

# ALERT

1.Display recent 50 disasters' alert in list function: no input(but since we are only doing PA this function will be same as 2. in disaster)

In [ ]:
result = SQL_Caller.disaster_top_50()

# ORDER (1 2 3 4 8 will be main goal)

Search order by id, display detailed info of that order
Function input:id
Return: detailed data of the order in any form (best list)
* Similar to the disaster but the function "sync_with_database" also supports overwrite id"

In [ ]:
test_order = Platform_Objects.Order()
test_order.sync_with_database(mydb, cursor, "ORDER_ID_EXAMPLE")

Search order by State, display list of all orders in that state(breif info: id, vendor, status)
Function input: state
Return: list of all orders(up to 50) of that state
* Of course, default is PA.

In [ ]:
order_result1 = SQL_Caller.order_top_50("CA")

Search order by vendor, display list of all orders of that vendor(breif info: id, vendor, status)
Function input: (Vendor id)
Return: list of all orders(up to 50) of that vendor

In [ ]:
order_result2 = SQL_Caller.order_by_vendor(mydb, cursor, "Vendor_ID_EXAMPLE")

Search order by status, display list of all orders of that status(completed, uncompleted, canceled etc..)
Function input: order status,
Return: list of all orders(up to 50) of that status
* Have not defined the status yet, did not limit the amount

In [ ]:
order_result3 = SQL_Caller.search_obj(mydb, cursor, 'Order', 'status = [completed/uncompleted/canceled]')

Search order by State and Vendor return list of all orders(up to 50) of that state and vendor

In [ ]:
order_result4 = SQL_Caller.order_state_vendor(mydb, cursor, "Vendor_ID_EXAMPLE", "NY")

Search order by State and status return list of all orders(up to 50) of that state and status

In [ ]:
order_result5 = SQL_Caller.order_status_state(mydb, cursor, 'status = [completed/uncompleted/canceled]', "MI")

Search order by Vendor and status return list of all orders(up to 50) of that vendor and status

In [ ]:
order_result6 = SQL_Caller.order_status_vendor(mydb, cursor, "Vendor_ID_EXAMPLE", 'status = [completed/uncompleted/canceled]')

Search order by State and Vendor and status return list of all orders(up to 50) of that state and vendor and status

In [ ]:
order_result7 = SQL_Caller.order_state_vendor_status(mydb, cursor, "Vendor_ID_EXAMPLE", 'status = [completed/uncompleted/canceled]', "OR")

Create Order and get the order id to manage shipments later:

In [ ]:
new_order = main.create_order("DISASTER ID")

# Create Shipment
Store order in database, display the shipment id for reference
function input: Order ID, State, Vendor(Can be blank), Supply Type, Amount.
Function will record the order and create an order id for that
return: shipment id(best named by date+name of supply type)
* The best way to do this is, assume this is for a disaster:

In [ ]:
destination_state = test_disaster.state_code
destination_county = test_disaster.county_code
new_shipment = Platform_Objects.Shipment(vendor="Vendor ID if determined", destination_state=destination_state, destination_county=destination_county, status="Not completed", vehicle_type="Vehicle ID if determined", current_location_lon=None, current_location_lat=None, content_id="Content/Supply ID", content_quantity="Quantity in number",
                 exp_date="Set a expiration date if desired")
# If any change before upload, try the following way to edit:
new_shipment.content_id = 1006
shipment_id = new_shipment.insert_to_database(mydb, cursor) # Also available with new_shipment.id after this command

For corresponding vendor id and name of each supply/content, assume know the content ID:

In [ ]:
generators = Platform_Objects.Content(1006) #Content ID for generators
generators.load_from_database(mydb, cursor)
vendors_for_generators = generators.get_cor_vendors(mydb, cursor)

Send request and route to that vendor(if vendor is filled), or send request to all vendor with the supplies
function: send request of the order and route to vendors
function input: same as 1.
return all vendor requested

* Since route need details of starting location, this is not developed yet. But email about the shipment is available

In [ ]:
new_shipment.notify(mydb, cursor)

* Destination zipcode is available via:

In [ ]:
destin_zip = SQL_Caller.fips_to_zip(mydb, cursor, new_shipment.destination_state, destination_county)[0]

Suggest order by disaster id, using our database to suggest order
function input: disaster id,
return the most recent order for disaster id,
if none, search for the most recent order for that type of disaster,
if still none return empty order
* Note that the relation has been established, waiting to bind with shipments

In [ ]:
next_order = main.suggest_order("DISASTER ID")

To get a list of all shipment related to an order:

In [ ]:
sub_shipments1 = new_order.shipment_list(mydb, cursor)

# Tracking
Track with order id, display the status of tracking on map
function input: order id,
return tracking messages(Accepted+time, Delivering+time, Delivered+time) for all shipment id relate with that order id
* No such info is available yet, so the current structure can only provide the coordinates:

In [ ]:
sub_shipments_locations = new_order.shipment_locations(mydb, cursor)

Track with shipment id, display the current position of trucks relate with the id
function  input: shipment id,
return mapping app interface OR map images with those trucks’ current location

In [ ]:
track_shipment = Platform_Objects.Shipment(s_id=shipment_id)
track_shipment.sync_with_database(mydb, cursor)
current_long = track_shipment.current_location_lon
current_lat = track_shipment.current_location_lat

Track by State: display a list of all accepted orders in that state
function input: State,
return list of all accepted orders in that state
* No actions taken yet

Track by Vendor: display a list of all accepted orders of that vendor
function input: vendor,
return list of all accepted orders of that vendor
* No actions taken yet

Track by supply type: display a list of all accepted order of that supply
function input: supply type,
return list of all accepted orders of that type
* No actions taken yet